# Validate Analysis Workflow Notebook
This notebook is a local version of the Google Colab-hosted analysis workflow notebook for MapSwipe Validate projects. This is for local integration testing of notebook / codebase interactions.

In [ ]:
%load_ext autoreload
%autoreload 2

# Step 1: Analyze Project

Replace the `project_id` value in the cell below with the ID of your project. Then open the Outline tab on the left-hand side of the screen, right-click on the `Step 1: Analyze Project` header, and choose `Select and Run Cell(s) for this Heading`.

In [ ]:
project_id = "-O7hFcC2pKTnTh01SGds"  # TODO choose a more friendly default

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from mapswipe.workflows.project_remap import analyze_project
analysis_results = analyze_project(project_id)

# Step 2: Visualize Results

In [ ]:
from mapswipe.workflows.viz import create_moran_quad_map, create_moran_quad_hex_map
from mapswipe.workflows.project_remap import MORAN_DISTANCE_METERS, DEPENDENT_VAR, MODEL_FEATURES, HEX_VIZ_H3_RESOLUTION

In [ ]:
df_agg_moran_w = analysis_results["df_agg_moran_w"]
#create_moran_quad_hex_map(df_agg_moran_w, f"moran_quad_{int(MORAN_DISTANCE_METERS)}m", value_cols=MODEL_FEATURES + ["remap_score", DEPENDENT_VAR], h3_resolution=HEX_VIZ_H3_RESOLUTION)
create_moran_quad_map(df_agg_moran_w, f"moran_quad_{int(MORAN_DISTANCE_METERS)}m", value_cols=MODEL_FEATURES + ["remap_score", DEPENDENT_VAR])

# Step 3: Set Thresholds
TODO Continue here